In [2]:
from bs4 import BeautifulSoup
import json
import re
import requests
from python_graphql_client import GraphqlClient
import time
import pandas as pd
import re
import numpy as np
from collections import Counter

In [2]:
client = GraphqlClient(endpoint="https://medium.com/_/graphql")

In [14]:
query = "query TopicHandler($topicSlug: ID!, $feedPagingOptions: PagingOptions, $sidebarPagingOptions: PagingOptions) {\n  topic(slug: $topicSlug) {\n    canonicalSlug\n    ...TopicScreen_topic\n    __typename\n  }\n}\n\nfragment PostListingItemFeed_postPreview on PostPreview {\n  post {\n    ...PostListingItemPreview_post\n    ...PostListingItemByline_post\n    ...PostListingItemImage_post\n    ...PostPresentationTracker_post\n    __typename\n  }\n  __typename\n}\n\nfragment PostListingItemPreview_post on Post {\n  id\n  mediumUrl\n  title\n  previewContent {\n    subtitle\n    isFullContent\n    __typename\n  }\n  isPublished\n  creator {\n    id\n    __typename\n  }\n  __typename\n}\n\nfragment PostListingItemByline_post on Post {\n  id\n  creator {\n    id\n    username\n    name\n    __typename\n  }\n  isLocked\n  readingTime\n  ...BookmarkButton_post\n  firstPublishedAt\n  updatedAt\n statusForCollection\n  collection {\n    id\n    name\n    ...collectionUrl_collection\n    __typename\n  }\n  __typename\n}\n\nfragment BookmarkButton_post on Post {\n  ...SusiClickable_post\n  ...WithSetReadingList_post\n  __typename\n}\n\nfragment SusiClickable_post on Post {\n  id\n  mediumUrl\n  ...SusiContainer_post\n  __typename\n}\n\nfragment SusiContainer_post on Post {\n  id\n  __typename\n}\n\nfragment WithSetReadingList_post on Post {\n  ...ReadingList_post\n  __typename\n}\n\nfragment ReadingList_post on Post {\n  __typename\n  id\n  readingList\n}\n\nfragment collectionUrl_collection on Collection {\n  id\n  domain\n  slug\n  __typename\n}\n\nfragment PostListingItemImage_post on Post {\n  id\n  mediumUrl\n  previewImage {\n    id\n    focusPercentX\n    focusPercentY\n    __typename\n  }\n  __typename\n}\n\nfragment PostPresentationTracker_post on Post {\n  id\n  visibility\n  previewContent {\n    isFullContent\n    __typename\n  }\n  collection {\n    id\n    __typename\n  }\n  __typename\n}\n\nfragment TopicScreen_topic on Topic {\n  id\n  ...TopicMetadata_topic\n  ...TopicLandingHeader_topic\n  ...TopicFeaturedAndLatest_topic\n  ...TopicLandingRelatedTopics_topic\n  ...TopicLandingPopular_posts\n  __typename\n}\n\nfragment TopicMetadata_topic on Topic {\n  name\n  description\n  slug\n  image {\n    id\n    __typename\n  }\n  __typename\n}\n\nfragment TopicLandingHeader_topic on Topic {\n  name\n  description\n  visibility\n  ...TopicFollowButtonSignedIn_topic\n  ...TopicFollowButtonSignedOut_topic\n  __typename\n}\n\nfragment TopicFollowButtonSignedIn_topic on Topic {\n  __typename\n  slug\n  isFollowing\n}\n\nfragment TopicFollowButtonSignedOut_topic on Topic {\n  id\n  slug\n  ...SusiClickable_topic\n  __typename\n}\n\nfragment SusiClickable_topic on Topic {\n  ...SusiContainer_topic\n  __typename\n}\n\nfragment SusiContainer_topic on Topic {\n  ...SignInOptions_topic\n  ...SignUpOptions_topic\n  __typename\n}\n\nfragment SignInOptions_topic on Topic {\n  id\n  name\n  __typename\n}\n\nfragment SignUpOptions_topic on Topic {\n  id\n  name\n  __typename\n}\n\nfragment TopicFeaturedAndLatest_topic on Topic {\n  name\n  slug\n  featuredPosts {\n    postPreviews {\n      post {\n        id\n        ...TopicLandingFeaturedStory_post\n        __typename\n      }\n      __typename\n    }\n    __typename\n  }\n  featuredTopicWriters(limit: 1) {\n    ...FeaturedWriter_featuredTopicWriter\n    __typename\n  }\n  latestPosts(paging: $feedPagingOptions) {\n    postPreviews {\n      post {\n        id\n        __typename\n      }\n      ...PostListingItemFeed_postPreview\n      __typename\n    }\n    pagingInfo {\n      next {\n        limit\n        to\n        __typename\n      }\n      __typename\n    }\n    __typename\n  }\n  __typename\n}\n\nfragment TopicLandingFeaturedStory_post on Post {\n  ...FeaturedPostPreview_post\n  ...PostListingItemPreview_post\n  ...PostListingItemBylineWithAvatar_post\n  ...PostListingItemImage_post\n  ...PostPresentationTracker_post\n  __typename\n}\n\nfragment FeaturedPostPreview_post on Post {\n  id\n  title\n  mediumUrl\n  previewContent {\n    subtitle\n    isFullContent\n    __typename\n  }\n  __typename\n}\n\nfragment PostListingItemBylineWithAvatar_post on Post {\n  creator {\n    username\n    name\n    id\n    imageId\n    mediumMemberAt\n    __typename\n  }\n  isLocked\n  readingTime\n  updatedAt\n  statusForCollection\n  collection {\n    id\n    name\n    ...collectionUrl_collection\n    __typename\n  }\n  __typename\n}\n\nfragment FeaturedWriter_featuredTopicWriter on FeaturedTopicWriter {\n  user {\n    id\n    username\n    name\n    bio\n    ...UserAvatar_user\n    ...UserFollowButton_user\n    __typename\n  }\n  posts {\n    postPreviews {\n      ...PostListingItemFeaturedWriter_postPreview\n      __typename\n    }\n    __typename\n  }\n  __typename\n}\n\nfragment UserAvatar_user on User {\n  username\n  id\n  name\n  imageId\n  mediumMemberAt\n  __typename\n}\n\nfragment UserFollowButton_user on User {\n  ...UserFollowButtonSignedIn_user\n  ...UserFollowButtonSignedOut_user\n  __typename\n}\n\nfragment UserFollowButtonSignedIn_user on User {\n  id\n  isFollowing\n  __typename\n}\n\nfragment UserFollowButtonSignedOut_user on User {\n  id\n  ...SusiClickable_user\n  __typename\n}\n\nfragment SusiClickable_user on User {\n  ...SusiContainer_user\n  __typename\n}\n\nfragment SusiContainer_user on User {\n  ...SignInOptions_user\n  ...SignUpOptions_user\n  __typename\n}\n\nfragment SignInOptions_user on User {\n  id\n  name\n  __typename\n}\n\nfragment SignUpOptions_user on User {\n  id\n  name\n  __typename\n}\n\nfragment PostListingItemFeaturedWriter_postPreview on PostPreview {\n  postId\n  post {\n    readingTime\n    id\n    mediumUrl\n    title\n    ...PostListingItemImage_post\n    ...PostPresentationTracker_post\n    __typename\n  }\n  __typename\n}\n\nfragment TopicLandingRelatedTopics_topic on Topic {\n  relatedTopics {\n    topic {\n      name\n      slug\n      __typename\n    }\n    __typename\n  }\n  __typename\n}\n\nfragment TopicLandingPopular_posts on Topic {\n  name\n  popularPosts(paging: $sidebarPagingOptions) {\n    postPreviews {\n      post {\n        ...PostListingItemSidebar_post\n        __typename\n      }\n      __typename\n    }\n    __typename\n  }\n  __typename\n}\n\nfragment PostListingItemSidebar_post on Post {\n  id\n  mediumUrl\n  title\n  readingTime\n  ...PostListingItemImage_post\n  ...PostPresentationTracker_post\n  __typename\n}\n"

variables = {"topicSlug": "popular", "feedPagingOptions": {"limit": 25, "to": f"{int(time.time()*1000)}"}, "sidebarPagingOptions": {"limit": 5}}

In [15]:
data = client.execute(query=query, variables=variables)

In [74]:
for post in data["data"]["topic"]["featuredPosts"]["postPreviews"]:
    print(f"{post['post']['updatedAt']} - {post['post']['title']} - {post['post']['mediumUrl']}")

1590519164019 - The Future of Movie Theaters Might Look a Lot Like an Apple Store - https://onezero.medium.com/the-future-of-movie-theaters-might-look-a-lot-like-an-apple-store-185c941d02a5
1589979986031 - Inside the Flour Company Supplying America’s Sudden Baking Obsession - https://marker.medium.com/inside-the-flour-company-supplying-americas-sudden-baking-obsession-623034583579
1590514172208 - Our Economy Was Just Blasted Years Into the Future - https://marker.medium.com/our-economy-was-just-blasted-years-into-the-future-a591fbba2298
1590471061357 - The Unique Frustration of Being Black and in Charge - https://level.medium.com/the-unique-frustration-of-being-black-and-in-charge-c1026bb5a9b3
1590384662134 - How Your Brain Prevents You From Getting Sick - https://elemental.medium.com/how-your-brain-prevents-you-from-getting-sick-201d8263344
1590384661084 - A Decision-Making Framework for Re-Entering the World - https://forge.medium.com/a-decision-making-framework-for-re-entering-the-w

In [85]:
def get_posts(data, posts):
    for post in data["data"]["topic"]["latestPosts"]["postPreviews"]:
        title = ""
        author = ""
        author_username = ""
        url = ""
        domain = ""

        if 'title' in post['post']:
            title = post['post']['title']
        if 'creator' in  post['post']:
            creator = post['post']['creator'] or []
            if 'username' in creator:
                author = post['post']['creator']['name']
                author_username = post['post']['creator']['username']
        if 'mediumUrl' in post['post']:
            url = post['post']['mediumUrl']
        if 'collection' in post['post']:
            collection = post['post']['collection'] or []
            if 'domain' in collection:
                domain = post['post']['collection']['domain']
        post = {
            "title": title, 
            "author": author,
            "author_username": author_username,
            "url": url,
            "domain": domain
        }
        if post not in posts:
            posts.append(post)
    return posts

In [86]:
posts = []
nb_h_per_day = 6
for i in range(365 * 24 // nb_h_per_day):
    variables = {"topicSlug": "popular", "feedPagingOptions": {"limit": 25, "to": f"{int(time.time() * 1000) - 3600000 * nb_h_per_day * i}"}, "sidebarPagingOptions": {"limit": 1}}
    data = client.execute(query=query, variables=variables)
    posts = get_posts(data, posts)

In [87]:
len(posts)

854

In [88]:
with open('data.json', "w") as f:
    json.dump(posts, f)

In [2]:
df = pd.read_json('data.json')
df.head()

,title,author,author_username,url,domain,tag
0,The Future of Movie Theaters Might Look a Lot ...,Eric Ravenscraft,lordravenscraft_50708,https://onezero.medium.com/the-future-of-movie...,onezero.medium.com,"[amc, movies, amazon, netflix, apple]"
1,Inside the Flour Company Supplying America’s S...,David H. Freedman,dhfreedman2,https://marker.medium.com/inside-the-flour-com...,marker.medium.com,"[king-arthur-flour, baking, business, consumer]"
2,Our Economy Was Just Blasted Years Into the Fu...,Steve LeVine,stevlevine,https://marker.medium.com/our-economy-was-just...,marker.medium.com,"[economy, business, society, future, automation]"
3,Joe Biden’s Latest “Gaffe” Has Left Me Almost ...,Lauren Martinchek,xLauren_Mx,https://medium.com/discourse/joe-bidens-latest...,None,[]
4,75 Things White People Can Do for Racial Justice,Corinne Shutack,cshutack,https://medium.com/equality-includes-you/what-...,None,[racism?source=post_page-----f2d18b0e0234-----...


In [7]:
def convert_to_num(s):
    if isinstance(s, int):
        return s
    elif 'K' in s:
        return int(1000 * float(s.replace('K','')))
    else:
        return int(s)

In [ ]:
for post in posts:
    post['tag'] = []
    post['claps'] = -1
    try:
        r = requests.get(post['url'])
    except:
        print(post['url'])
        continue
    soup = BeautifulSoup(r.text)
    # tags
    hrefs = soup.findAll('a', href=True)
    for href in hrefs:
        res = re.search("^/tagged/(.*)", href['href'])
        if res:
            post['tag'].append(res.group(1))
    # claps
    buttons = soup.findAll('button')
    for i, button in enumerate(buttons):
        res = re.search("(.*) claps", button.text)
        if res:
            post['claps'] = convert_to_num(res.group(1))

In [55]:
with open('data.json', "w") as f:
    json.dump(posts, f)

In [4]:
with open('data.json', "r") as f:
    posts = json.load(f)

In [5]:
df = pd.read_json('data.json')
df.head()

,title,author,author_username,url,domain,tag,claps
0,The Future of Movie Theaters Might Look a Lot ...,Eric Ravenscraft,lordravenscraft_50708,https://onezero.medium.com/the-future-of-movie...,onezero.medium.com,"[amc, movies, amazon, netflix, apple]",1.5K
1,Inside the Flour Company Supplying America’s S...,David H. Freedman,dhfreedman2,https://marker.medium.com/inside-the-flour-com...,marker.medium.com,"[king-arthur-flour, baking, business, consumer]",4.4K
2,Our Economy Was Just Blasted Years Into the Fu...,Steve LeVine,stevlevine,https://marker.medium.com/our-economy-was-just...,marker.medium.com,"[economy, business, society, future, automation]",3.6K
3,Joe Biden’s Latest “Gaffe” Has Left Me Almost ...,Lauren Martinchek,xLauren_Mx,https://medium.com/discourse/joe-bidens-latest...,None,[],5.9K
4,75 Things White People Can Do for Racial Justice,Corinne Shutack,cshutack,https://medium.com/equality-includes-you/what-...,None,[],13.8K


In [53]:
tags = []
wtags = []
for r,n in zip(df.tag, df.numclaps):
    tags.extend(r)
    wtags.extend(r * n)

In [58]:
unique_tags = np.unique(np.array(tags))

In [59]:
print(f"{len(tags)} - {len(unique_tags)}")

3019 - 906


In [54]:
c = Counter(tags)

In [55]:
c.most_common(20)

[('health', 107),
 ('culture', 73),
 ('coronavirus', 59),
 ('politics', 53),
 ('digital-life', 51),
 ('life', 48),
 ('industry', 47),
 ('body', 46),
 ('technology', 37),
 ('science', 36),
 ('tech', 36),
 ('relationships', 34),
 ('self-improvement', 33),
 ('mental-health', 32),
 ('business', 31),
 ('self', 31),
 ('psychology', 30),
 ('food', 29),
 ('brain', 28),
 ('data-science', 28)]

In [56]:
wc = Counter(wtags)
wc.most_common(20)

[('life-lessons', 789000),
 ('life', 784320),
 ('productivity', 777720),
 ('health', 737546),
 ('self-improvement', 658100),
 ('coronavirus', 645071),
 ('psychology', 644520),
 ('politics', 593697),
 ('mental-health', 538080),
 ('self', 527500),
 ('culture', 451832),
 ('inspiration', 438400),
 ('relationships', 367020),
 ('succeed', 320600),
 ('rethink', 267200),
 ('body', 266630),
 ('live', 247100),
 ('digital-life', 237861),
 ('economy', 230200),
 ('industry', 223428)]

In [8]:
df['numclaps'] = df.claps.apply(convert_to_num)

In [37]:
df_tech = df[df.tag.apply(lambda x: "technology" in x)]
df_tech.head()

,title,author,author_username,url,domain,tag,claps,numclaps
25,The Problem Isn’t Zoom Fatigue — It’s Mourning...,Evan Selinger,evanselinger,https://onezero.medium.com/the-problem-isnt-zo...,onezero.medium.com,"[video-conferencing, digital-life, culture, zo...",1.2K,1200
61,The Debate Over Screens and Health is More Con...,Markham Heid,mheidj,https://elemental.medium.com/kids-are-staring-...,elemental.medium.com,"[health, science, brain, parenting, technology]",508,508
69,CEO of Surveillance Firm Banjo Once Helped KKK...,Matt Stroud,stroudjournalism,https://onezero.medium.com/ceo-of-surveillance...,onezero.medium.com,"[surveillance, privacy, technology, racism, si...",2.9K,2900
93,Zoom Is a Nightmare. So Why Is Everyone Still ...,Simon Pitt,simon_pitt,https://onezero.medium.com/zoom-is-a-nightmare...,onezero.medium.com,"[zoom, video-conferencing, digital-life, techn...",23K,23000
97,We Live in the Blue Ruin,Felicia C. Sullivan,felsull,https://humanparts.medium.com/blue-ruin-ab4b6d...,humanparts.medium.com,"[fiction, dystopia, internet, technology, cult...",1.3K,1300


In [42]:
tech_tags = []
for r,n in zip(df_tech.tag, df_tech.numclaps):
    tech_tags.extend(r * n)

In [43]:
c_tech = Counter(tech_tags)
c_tech.most_common(10)

[('technology', 148447),
 ('digital-life', 43100),
 ('startup', 25900),
 ('video-conferencing', 24200),
 ('zoom', 24200),
 ('data-science', 23500),
 ('remote-working', 23000),
 ('artificial-intelligence', 22106),
 ('industry', 21900),
 ('programming', 16282)]

In [ ]:
popular_tags = "technology", "startup", "data-science", "artificial-intelligence", "programming"

In [6]:
day = "29"
month = "05"
year = "2020"
tag = "technology"
url = f"https://medium.com/tag/{tag}/archive/{year}/{month}/{day}"

In [ ]:
r = requests.get(url)

In [63]:
soup = BeautifulSoup(r.content)

In [67]:
soup.find_all('div', class_="postArticle")

[]

# https://github.com/collindching/Maximizing-Medium-Claps

In [8]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

options = Options()
options.add_argument('--headless')
options.add_argument('--disable-gpu')  # Last I checked this was necessary.

In [10]:
driver = webdriver.Chrome(options=options)
driver.get(url)
soup = BeautifulSoup(driver.page_source, 'lxml')

NameError: name 'BeautifulSoup' is not defined

In [ ]:
class ArchiveScraper():
    
    def __init__(self):
        self.scraper_class = 'ArchiveScraper' 
    
    def get_authors(self, soup):
        try:
            authors = []
            
            article_cards = soup.find_all('div',{'class': 'postArticle'})
            for card in article_cards:
                authors.append(card.select('a.ds-link')[0].text)
                
            return authors
        
        except:
            print("Couldn't get authors.")
    
    
    def get_publications(self, soup):
        try:
            publications = []
            
            article_cards = soup.find_all('div',{'class': 'postArticle'})
            for card in article_cards:        
                published_by_soup = card.select('a.ds-link')
    
                if len(published_by_soup) == 1:
                    publications.append('None')
                    continue

                publications.append(published_by_soup[1].text)

            return publications
        
        except:
            print("Couldn't get publications.")
    
    
    def get_titles(self, soup):
        try:
            titles = []
            
            article_cards = soup.find_all('div',{'class': 'postArticle'})
            for card in article_cards:
                title_soup = card.find('h3')

                if title_soup is None:
                    title = card.find('p').text
                    titles.append(title)
                else:
                    title = card.find('h3').text
                    titles.append(title)
            
            titles = [x.replace('\xa0',' ').replace('\u200a—\u200a','--').replace('--', ' — ') for x in titles]
            return titles
        
        except:
            print("Couldn't get titles.")
    
    
    def get_read_times(self, soup):
        try:
            read_times = []
            
            read_time_soups = soup.find_all('span', {'class': 'readingTime'})
            read_times = [int(x['title'].replace(" min read","")) for x in read_time_soups]
            
            return read_times
        
        except:
            print("Couldn't get read times.")    
            
            
    def count_responses(self, soup):
        try:
            responses = []

            article_cards = soup.find_all('div', {'class': 'postArticle'})
            for card in article_cards:
                responses_text = card.select('div.buttonSet.u-floatRight')[0].text
                if responses_text == '':
                    responses.append(0)
                else:
                    responses.append(int(responses_text.replace(' responses','').replace(' response','')))
        
            return responses
            
        except:
            print("Couldn't get responses.")
    
    
    def count_claps(self, soup):
        try:
            claps = []

            article_cards = soup.find_all('div', {'class': 'postArticle'})
            for card in article_cards:
                clap_soup = card.find('button',{'data-action':'show-recommends'})

                # no claps
                if clap_soup is None:
                    claps.append(0)
                # >1k claps
                elif 'K' in clap_soup.text:
                    claps.append(int(float(clap_soup.text.replace('K',''))*1000))
                # normal case
                else:
                    claps.append(int(clap_soup.text))
                    
            return claps
        
        except:
            print("Couldn't get claps.")
            
            
    def get_dates(self, soup):
        dates = []
        article_cards = soup.find_all('div',{'class': 'postArticle'})
        for i, card in enumerate(article_cards):
            try:
                dates.append(card.find('span',{'class':'middotDivider'}).previous_element)
            
            except:
                print("Couldn't get this date.")
                dates.append('skip')
                continue
                
        return dates
            
    def get_article_links(self, soup):
        try:
            links = []

            article_cards = soup.find_all('div',{'class': 'postArticle'})
            for card in article_cards:
                links.append(card.find('div', {'class': 'postArticle-content'}).parent['href'])

            return links
        
        except:
            print("Couldn't get article links.")
            
    
    def scrape(self, soup):
        archive_data = {
            'authors': self.get_authors(soup),
            'publications': self.get_publications(soup),
            'titles': self.get_titles(soup),
            'read_times': self.get_read_times(soup),
            'n_responses': self.count_responses(soup),
            'n_claps': self.count_claps(soup),
            'dates': self.get_dates(soup),
            'article_links': self.get_article_links(soup)
        }
        
        return archive_data

In [ ]:
class ArticleScraper():
    """
    Scrapes all data for an article and returns it in JSON format.
    """
    
    def __init__(self):
        self.scraper_class = 'ArticleScraper' 
    
    def get_title(self, soup):
        try:
            title = soup.find('h1').text
            return title
        
        except:
            print("Couldn't get title from article.")
        
    def get_subtitle(self, soup):
        try:
            subtitle_soup = soup.find('h1').parent.parent.next_sibling.find('h2')
            subtitle = clean_string(subtitle_soup.text)
            return subtitle
        
        except:
            print("Couldn't (or didn't) get subtitle from article.") 
            return 'None'
            
    def get_tags(self, soup):
        try:
            tags = []
            tags = [x.text for x in soup.find_all("a", href=re.compile(".*tag.*"))]
            if len(tags) == 0:
                return 'None'
            else:
                return tags
            
        except:
            print("Couldn't get article tags.")
    
    def get_author(self, soup):
        try:
            author = soup.find('div',style=re.compile(r'flex:.*')).find('a').text
            return author
        except:
            print("Couldn't get author from article.")
            
    def get_h1_headers(self, soup):
        try:
            article_soup = soup.find('article')
            h1_header_soups = article_soup.find_all('h1')
            
            if len(h1_header_soups) == 1:
                return 'None'
            else:
                h1_headers = [clean_string(x.text) for x in h1_header_soups[1:]]
                return h1_headers
        except:
            print("Couldn't get h1 headers.")
            
    
    def get_h2_headers(self, soup):
        try:
            h2_headers = []
            
            article_soup = soup.find('article')
            h2_header_soups = article_soup.find_all('h2')
            
            for header_soup in h2_header_soups:
                if header_soup.text == "Dive in. We'll learn what you like along the way.":
                    continue
                else:
                    h2_headers.append(clean_string(header_soup.text))
            
            if len(h2_headers) == 0:
                return 'None'
            else:
                return h2_headers
        
        except:
            print("Couldn't get h2 headers.")
            
    def get_paragraphs(self, soup):
        try:
            article_soup = soup.find('article')
            paragraphs = [clean_string(x.text) for x in article_soup.find_all('p')]
            return paragraphs
        except:
            print("Couldn't scrape paragraphs.")
            
    def get_blockquotes(self, soup):
        try:
            article_soup = soup.find('article')
            blockquotes = [x.text for x in article_soup.find_all('blockquote')]
            return blockquotes
        except:
            print("Couldn't (or didn't) find blockquotes.")
            return 'None'
            
    def get_bolded(self,soup):
        try:
            article_soup = soup.find('article')

            bolded = [x.text.strip() for x in article_soup.find_all('strong')]
            bolded = [clean_string(x) for x in bolded if filter_string(x)]
            bolded = [x for x in bolded if x]
          
            if len(bolded) == 0:
                return 'None'
            else:
                return bolded
        except:
            print("Couldn't get bolded text.")
    
    def get_italics(self, soup):
        try:
            article_soup = soup.find('article')
            italics = [x.text.strip() for x in article_soup.find_all('em')]
            italics = [clean_string(x) for x in italics if filter_string(x)]
            
            if len(italics) == 0:
                return 'None'
            else:
                return italics
        except:
            print("Couldnt get italicized text.")
            
    def count_bullet_lists(self, soup):
        try:
            article_soup = soup.find('article')
            return len(article_soup.find_all('ul'))
        
        except:
            print("Couldn't count bullet lists.")
            
    def count_numbered_lists(self, soup):
        try:
            article_soup = soup.find('article')
            return len(article_soup.find_all('ol'))
        
        except:
            print("Couldn't count numbered lists.")
            
    def count_figures(self, soup):
        try:
            article_soup = soup.find('article')
            figures = article_soup.find_all('figure')
            return len(figures)
        
        except:
            print("Couldn't count images.")  
            
    def count_gists(self, soup):
        try:
            gists = []
            article_soup = soup.find('article')
            for fig in article_soup.find_all('figure'):
                gist_soup = fig.find('iframe', title=re.compile('.*\.py'))
                if gist_soup == None:
                    continue
                else:
                    gists.append(gist_soup)
                    
            return len(gists)
        
        except:
            print("Couldn't count gists.")
            
    def count_code_chunks(self, soup):
        try:
            article_soup = soup.find('article')
            code_chunk_soups = article_soup.find_all('pre')
            return len(code_chunk_soups)
        
        except:
            print("Couldn't count code chunks.")
            
    def count_vids(self, soup):
        try:
            yt_vids = []
            article_soup = soup.find('article')
            for figure in article_soup.find_all('figure'):
                yt_soup = figure.find('iframe', src=re.compile('.*youtube.*'))
                if yt_soup == None:
                    continue
                else:
                    yt_vids.append(yt_soup)
                    
            return len(yt_vids)
                    
        except:
            print("Couldn't get YouTube videos.")        
            
    def count_links(self, soup):
        try:
            article_soup = soup.find('article')
            link_soups = article_soup.find_all('a', {'target': '_blank'})
            return len(link_soups)
        
        except:
            print("Couldn't count links.")      
            
    def scrape(self, soup):
        article_data = {
            "title": self.get_title(soup),
            "subtitle": self.get_subtitle(soup),
            "tags": self.get_tags(soup),
            "author": self.get_author(soup),
            "h1_headers": self.get_h1_headers(soup),
            "h2_headers": self.get_h2_headers(soup),
            "paragraphs": self.get_paragraphs(soup),
            "blockquotes": self.get_blockquotes(soup),
            "bold_text": self.get_bolded(soup),
            "italic_text": self.get_italics(soup),
            "n_figures": self.count_figures(soup),
            "n_bullet_lists": self.count_bullet_lists(soup),
            "n_numbered_lists": self.count_numbered_lists(soup),
            "n_gists": self.count_gists(soup),
            "n_code_chunks": self.count_code_chunks(soup),
            "n_vids": self.count_vids(soup),
            "n_links": self.count_links(soup),
        }
        
        # if subtitle exists, remove it from h2_headers list
        subtitle = article_data['subtitle']
        if subtitle != 'None':
            article_data['h2_headers'].remove(subtitle)
            
            if len(article_data['h2_headers']) == 0:
                article_data['h2_headers'] = 'None'
                
        return(article_data)